In [1]:
import pandas as pd
import sqlite3

In [2]:
!gdown 1rOjHdkmEvwlyIWCN1vNBnNfonaU7ajbf

Downloading...
From: https://drive.google.com/uc?id=1rOjHdkmEvwlyIWCN1vNBnNfonaU7ajbf
To: /home/iqbal/playGround/tech-testDs/e_commerce_transactions.csv
100%|████████████████████████████████████████| 372k/372k [00:00<00:00, 2.08MB/s]


In [3]:
df = pd.read_csv('e_commerce_transactions.csv')
df.head(3)

,order_id,customer_id,order_date,payment_value,decoy_flag,decoy_noise
0,101000,684,2024-01-29,38.93,A,39.51
1,101001,207,2024-06-29,59.52,B,38.97
2,101002,288,2024-03-20,355.52,B,338.14


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       10000 non-null  int64  
 1   customer_id    10000 non-null  int64  
 2   order_date     10000 non-null  object 
 3   payment_value  10000 non-null  float64
 4   decoy_flag     10000 non-null  object 
 5   decoy_noise    10000 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 468.9+ KB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_id,10000.0,105999.500000,2886.895680,101000.00,103499.7500,105999.500,108499.2500,110999.00
customer_id,10000.0,499.053900,289.171477,1.00,245.7500,502.000,746.0000,1000.00
payment_value,10000.0,201.130120,141.395236,0.55,96.3775,169.025,271.1425,1369.47
decoy_noise,10000.0,201.107939,146.458254,-48.34,96.4225,167.240,271.1250,1468.46


In [6]:
df['decoy_flag'].unique()

array(['A', 'B', 'C', 'D'], dtype=object)

In [7]:
con = sqlite3.connect('analisis.sqlite')
cur = con.cursor()

In [8]:
df.to_sql('e_commers_transactions', con=con, if_exists='replace')

10000

In [9]:
rfm_query = """
    SELECT
        customer_id,
        JULIANDAY('now') - JULIANDAY(MAX(order_date)) AS recency,
        COUNT(order_id) AS frequency,
        SUM(payment_value) AS monetary
    FROM e_commers_transactions
    GROUP BY customer_id
"""
rfm_df = pd.read_sql_query(rfm_query, con)

In [26]:
scoring = """
-- Hitung Recency, Frequency, Monetary
WITH base AS (
  SELECT 
    customer_id,
    MAX(order_date) AS last_order,
    COUNT(order_id) AS frequency,
    SUM(payment_value) AS monetary
  FROM e_commers_transactions
  GROUP BY customer_id
),
snapshot AS (
  SELECT DATE(MAX(order_date), '+1 day') AS snapshot_date
  FROM e_commers_transactions
),
rfm AS (
  SELECT
    b.customer_id,
    CAST(JULIANDAY(s.snapshot_date) - JULIANDAY(b.last_order) AS INTEGER) AS recency,
    b.frequency,
    b.monetary
  FROM base b
  CROSS JOIN snapshot s
)
SELECT * FROM rfm;

"""
pd.read_sql_query(scoring, con)

,customer_id,recency,frequency,monetary
0,1,433,6,517.09
1,2,411,14,3198.96
2,3,198,10,1892.63
3,4,390,9,1232.10
4,5,288,12,3372.05
...,...,...,...,...
995,996,326,7,1688.47
996,997,167,13,3121.41
997,998,319,12,1665.19
998,999,143,11,2019.28


In [27]:
final = """
-- Tambahkan segmentasi skor R, F, M secara manual
WITH rfm AS (
  SELECT
    customer_id,
    CAST(JULIANDAY((SELECT MAX(order_date) FROM e_commerce_transactions)) - JULIANDAY(MAX(order_date)) AS INTEGER) AS recency,
    COUNT(order_id) AS frequency,
    SUM(payment_value) AS monetary
  FROM e_commerce_transactions
  GROUP BY customer_id
),
scored AS (
  SELECT *,
    CASE
      WHEN recency <= 30 THEN 3
      WHEN recency <= 90 THEN 2
      ELSE 1
    END AS r_score,
    CASE
      WHEN frequency >= 10 THEN 3
      WHEN frequency >= 5 THEN 2
      ELSE 1
    END AS f_score,
    CASE
      WHEN monetary >= 1000 THEN 3
      WHEN monetary >= 500 THEN 2
      ELSE 1
    END AS m_score
  FROM rfm
),
final AS (
  SELECT *,
    (r_score + f_score + m_score) AS rfm_score
  FROM scored
)
SELECT *,
  CASE
    WHEN rfm_score >= 8 THEN 'High Value'
    WHEN rfm_score >= 7 THEN 'Mid-High Value'
    WHEN rfm_score >= 6 THEN 'Mid Value'
    WHEN rfm_score >= 5 THEN 'Mid-Low Value'
    ELSE 'Low Value'
  END AS segment
FROM final;

"""
pd.read_sql_query(final, con)

DatabaseError: Execution failed on sql '
-- Tambahkan segmentasi skor R, F, M secara manual
WITH rfm AS (
  SELECT
    customer_id,
    CAST(JULIANDAY((SELECT MAX(order_date) FROM e_commerce_transactions)) - JULIANDAY(MAX(order_date)) AS INTEGER) AS recency,
    COUNT(order_id) AS frequency,
    SUM(payment_value) AS monetary
  FROM e_commerce_transactions
  GROUP BY customer_id
),
scored AS (
  SELECT *,
    CASE
      WHEN recency <= 30 THEN 3
      WHEN recency <= 90 THEN 2
      ELSE 1
    END AS r_score,
    CASE
      WHEN frequency >= 10 THEN 3
      WHEN frequency >= 5 THEN 2
      ELSE 1
    END AS f_score,
    CASE
      WHEN monetary >= 1000 THEN 3
      WHEN monetary >= 500 THEN 2
      ELSE 1
    END AS m_score
  FROM rfm
),
final AS (
  SELECT *,
    (r_score + f_score + m_score) AS rfm_score
  FROM scored
)
SELECT *,
  CASE
    WHEN rfm_score >= 8 THEN 'High Value'
    WHEN rfm_score >= 7 THEN 'Mid-High Value'
    WHEN rfm_score >= 6 THEN 'Mid Value'
    WHEN rfm_score >

In [23]:
# Add manual scoring
rfm_df['recency_score'] = pd.qcut(rfm_df['recency'], 3, labels=[3,2,1]).astype(int)
rfm_df['frequency_score'] = pd.qcut(rfm_df['frequency'].rank(method='first'), 3, labels=[1,2,3]).astype(int)
rfm_df['monetary_score'] = pd.qcut(rfm_df['monetary'].rank(method='first'), 3, labels=[1,2,3]).astype(int)
rfm_df['rfm_score'] = rfm_df[['recency_score','frequency_score','monetary_score']].sum(axis=1)
rfm_df

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score
0,1,475.660627,6,517.09,1,1,1,3
1,2,453.660627,14,3198.96,1,3,3,7
2,3,240.660627,10,1892.63,3,2,2,7
3,4,432.660627,9,1232.10,1,2,1,4
4,5,330.660627,12,3372.05,3,3,3,9
...,...,...,...,...,...,...,...,...
995,996,368.660627,7,1688.47,2,1,2,5
996,997,209.660627,13,3121.41,3,3,3,9
997,998,361.660627,12,1665.19,2,3,2,7
998,999,185.660627,11,2019.28,3,3,2,8


In [11]:
flag_summary = pd.read_sql_query("""
    SELECT decoy_flag, COUNT(*) AS count, AVG(payment_value) AS avg_payment,
           MAX(payment_value) AS max_payment, MIN(payment_value) AS min_payment
    FROM e_commers_transactions
    GROUP BY decoy_flag
""", con)

In [12]:
print("Decoy flag summary:")
print(flag_summary)

Decoy flag summary:
  decoy_flag  count  avg_payment  max_payment  min_payment
0          A   1486   194.865121       947.08         0.55
1          B   2519   201.876038      1369.47         1.35
2          C   4936   203.126078      1021.76         1.97
3          D   1059   198.843787       889.26         6.98


In [13]:
op_noise = pd.read_sql_query("""
    SELECT * FROM e_commers_transactions
    ORDER BY decoy_noise DESC
    LIMIT 10
""", con)
op_noise

,index,order_id,customer_id,order_date,payment_value,decoy_flag,decoy_noise
0,6006,107006,501,2024-02-13,1369.47,B,1468.46
1,9717,110717,930,2024-01-20,1021.76,C,1170.73
2,8909,109909,880,2024-09-14,947.08,A,1153.43
3,2012,103012,309,2024-05-22,967.38,C,1144.52
4,8462,109462,16,2024-01-19,988.11,B,1095.57
5,9822,110822,521,2024-03-09,913.93,A,1088.74
6,3854,104854,22,2024-03-06,861.75,C,1041.10
7,6161,107161,763,2024-01-22,817.19,C,1011.15
8,151,101151,408,2024-01-17,928.50,B,995.56
9,6401,107401,379,2024-05-08,847.07,C,984.28


In [14]:
anomalies = pd.read_sql_query("""
    SELECT * FROM e_commers_transactions
    WHERE decoy_noise > 10000
""", con)

In [15]:
anomalies

,index,order_id,customer_id,order_date,payment_value,decoy_flag,decoy_noise


In [16]:
con.execute("SELECT * FROM e_commers_transactions where decoy_noise > 1400").fetchall()

[(6006, 107006, 501, '2024-02-13', 1369.47, 'B', 1468.46)]

In [17]:
# Repeat purchase
repeat_query = """
    SELECT STRFTIME('%Y-%m', order_date) AS month,
           COUNT(DISTINCT customer_id) AS unique_customers,
           COUNT(customer_id) AS total_orders
    FROM e_commers_transactions
    GROUP BY month
    HAVING unique_customers > 1
"""
repeat_df = pd.read_sql_query(repeat_query, con)
print(repeat_df.head())

     month  unique_customers  total_orders
0  2024-01               980          4095
1  2024-02               895          2324
2  2024-03               763          1449
3  2024-04               563           845
4  2024-05               388           506


In [18]:
# Detect anomalies using IQR for decoy_noise
Q1 = df['decoy_noise'].quantile(0.25)
Q3 = df['decoy_noise'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
anomalies_noise = df[df['decoy_noise'] > upper_bound]

# Detect anomalies using distribution of decoy_flag
flag_summary = df.groupby("decoy_flag")["payment_value"].agg(["count", "mean", "std"])
print("Decoy flag distribution:\n", flag_summary)

# Combine both anomaly signals
decoy_flag_outliers = flag_summary[flag_summary['mean'] > flag_summary['mean'].mean() + 2*flag_summary['std']]
flag_suspects = df[df['decoy_flag'].isin(decoy_flag_outliers.index)]

# Final anomalies: high decoy_noise + suspicious decoy_flag
anomalies_combined = df[(df['decoy_noise'] > upper_bound) | (df['decoy_flag'].isin(decoy_flag_outliers.index))]

print("Combined anomalies:\n", anomalies_combined.head())

Decoy flag distribution:
             count        mean         std
decoy_flag                               
A            1486  194.865121  137.100823
B            2519  201.876038  143.637576
C            4936  203.126078  141.796813
D            1059  198.843787  139.986094
Combined anomalies:
     order_id  customer_id  order_date  payment_value decoy_flag  decoy_noise
23    101023          744  2024-01-28         714.54          A       688.41
33    101033          816  2024-06-27         611.16          B       727.26
49    101049          489  2024-01-13         548.90          C       624.02
57    101057          988  2024-01-14         645.70          C       730.71
70    101070          695  2024-03-29         617.81          C       679.00


In [19]:
anomaly_sql = """
    WITH noise_stats AS (
      SELECT
        AVG(decoy_noise) AS mean_noise,
        (MAX(decoy_noise) - MIN(decoy_noise)) AS range_noise
      FROM e_commers_transactions
    ),
    flag_stats AS (
      SELECT decoy_flag,
             COUNT(*) AS count,
             AVG(payment_value) AS mean_value
      FROM e_commers_transactions
      GROUP BY decoy_flag
    ),
    global_flag_avg AS (
      SELECT AVG(mean_value) AS avg_mean_flag
      FROM flag_stats
    )
    SELECT *
    FROM e_commers_transactions
    WHERE decoy_noise > (
      SELECT mean_noise + 0.5 * range_noise FROM noise_stats
    )
    OR decoy_flag IN (
      SELECT fs.decoy_flag
      FROM flag_stats fs
      JOIN global_flag_avg gfa
      ON fs.mean_value > gfa.avg_mean_flag * 2
    )
"""
anomaly = pd.read_sql_query(anomaly_sql, con)
print("SQL (consptual) anomaly query:\n", anomaly)

SQL (consptual) anomaly query:
    index  order_id  customer_id  order_date  payment_value decoy_flag  \
0    151    101151          408  2024-01-17         928.50          B   
1   2012    103012          309  2024-05-22         967.38          C   
2   3854    104854           22  2024-03-06         861.75          C   
3   6006    107006          501  2024-02-13        1369.47          B   
4   6161    107161          763  2024-01-22         817.19          C   
5   6401    107401          379  2024-05-08         847.07          C   
6   8462    109462           16  2024-01-19         988.11          B   
7   8909    109909          880  2024-09-14         947.08          A   
8   9717    110717          930  2024-01-20        1021.76          C   
9   9822    110822          521  2024-03-09         913.93          A   

   decoy_noise  
0       995.56  
1      1144.52  
2      1041.10  
3      1468.46  
4      1011.15  
5       984.28  
6      1095.57  
7      1153.43  
8      1170